# Attempt using ColorThief

In [ ]:
from colorthief import ColorThief
color_thief = ColorThief('sample_image.jpg')
# get the dominant color
dominant_color = color_thief.get_color(quality=1)

In [ ]:
color_thief.get_palette(quality=10, color_count = 10)

In [ ]:
RGB2HEX(dominant_color)

# Another Way

In [1]:
import cv2
import numpy as np
from skimage import io

In [ ]:
img = io.imread('sample_image.png')[:, :, :-1]

average = img.mean(axis=0).mean(axis=0)

pixels = np.float32(img.reshape(-1, 3))

n_colors = 5
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
_, counts = np.unique(labels, return_counts=True)


dominant = palette[np.argmax(counts)]

average = img.mean(axis=0).mean(axis=0)

pixels = np.float32(img.reshape(-1, 3))

n_colors = 5
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
_, counts = np.unique(labels, return_counts=True)


dominant = palette[np.argmax(counts)]

In [ ]:
import matplotlib.pyplot as plt

avg_patch = np.ones(shape=img.shape, dtype=np.uint8)*np.uint8(average)

indices = np.argsort(counts)[::-1]   
freqs = np.cumsum(np.hstack([[0], counts[indices]/counts.sum()]))
rows = np.int_(img.shape[0]*freqs)

dom_patch = np.zeros(shape=img.shape, dtype=np.uint8)
for i in range(len(rows) - 1):
    dom_patch[rows[i]:rows[i + 1], :, :] += np.uint8(palette[indices[i]])

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12,6))
ax0.imshow(avg_patch)
ax0.set_title('Average color')
ax0.axis('off')
ax1.imshow(dom_patch)
ax1.set_title('Dominant colors')
ax1.axis('off')
plt.show(fig)


In [ ]:
import cv2

# Opens the Video file
cap= cv2.VideoCapture('sample_video.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('vid_cap/' + str(i) + '.png',frame)
    i+=1

cap.release()
cv2.destroyAllWindows()

In [2]:
from tqdm import tqdm_notebook as tqdm
import glob

In [74]:
dominant_colors = {}
average_colors = {}

filelist = glob.iglob('vid_cap/*.png')

for filepath in sorted(filelist):
    img = io.imread(filepath)[:, :, :-1]

    pixels = np.float32(img.reshape(-1, 3))

    n_colors = 5
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)

    dominant = palette[np.argmax(counts)]
    
    myimg = cv2.imread(filepath)
    avg_color_per_row = numpy.average(myimg, axis=0)
    avg_color = numpy.average(avg_color_per_row, axis=0)
    
    dominant_colors[filepath[8:].split('.')[0]] = dominant
    average_colors[filepath[8:].split('.')[0]] = avg_color

In [4]:
import pandas as pd

In [105]:
df_d = pd.DataFrame.from_dict(dominant_colors).transpose()
df_a = pd.DataFrame.from_dict(average_colors).transpose()

In [106]:
df_d.reset_index(inplace = True)
df_a.reset_index(inplace = True)

In [107]:
df_d['index'] = pd.to_numeric(df_d['index'])
df_a['index'] = pd.to_numeric(df_a['index'])

In [111]:
df_d.sort_values('index', inplace = True)
df_d.reset_index(drop = True, inplace = True)

df_a.sort_values('index', inplace = True)
df_a.reset_index(drop = True, inplace = True)

In [124]:
for i in range(len(df_d)):
    # Create a blank 300x300 black image
    image = np.zeros((300, 300, 3), np.uint8)
    # Fill image with red color(set each pixel to red)
    image[:] = (df_d[0][i], df_d[1][i], df_d[2][i])
    if i < 10:
        cv2.imwrite('dom_out/d00' + str(i) + '.png', image)
    elif 9 < i < 100:
        cv2.imwrite('dom_out/d0' + str(i) + '.png', image)
    else:
        cv2.imwrite('dom_out/d' + str(i) + '.png', image)
    
for i in range(len(df_a)):
    # Create a blank 300x300 black image
    image = np.zeros((300, 300, 3), np.uint8)
    # Fill image with red color(set each pixel to red)
    image[:] = (df_a[0][i], df_a[1][i], df_a[2][i])
    if i < 10:
        cv2.imwrite('avg_out/a00' + str(i) + '.png', image)
    elif 9 < i < 100:
        cv2.imwrite('avg_out/a0' + str(i) + '.png', image)
    else:
        cv2.imwrite('avg_out/a' + str(i) + '.png', image)


# Make Video

In [115]:
import os

In [145]:
image_folder = 'avg_out'
video_name = 'avg_out.avi'

images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, frameSize = (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()